In [1]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llama-3.2-11b-text-preview",
    messages=[
        {
            "role": "user",
            "content": "HI\n"
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


How can I assist you today?

In [6]:
from groq import Groq
from duckduckgo_search import DDGS
from py_expression_eval import Parser
import re, time

# Initialize Groq client
client = Groq()

# Initialize calculator
parser = Parser()

# DuckDuckGo search function
def search(search_term):
    search_result = ""
    with DDGS() as ddgs:
        results = ddgs.text(search_term, max_results=5)
        for result in results:
            search_result += result['body'] + " "
    return search_result

# Calculator function
def calculator(str):
    return parser.parse(str).evaluate({})

# System prompt
SYSTEM_PROMPT = """
Answer the following questions and obey the following commands as best you can.

You have access to the following tools:

Search: useful for when you need to answer questions about current events. You should ask targeted questions.
Calculator: Useful for when you need to answer questions about math. Use python code, eg: 2 + 2
Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of two things:

Option 1: You use a tool to answer the question.
For this, you should use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: "the input to the action, to be sent to the tool"

After this, the human will respond with an observation, and you will continue.

Option 2: You respond to the human.
For this, you should use the following format:
Action: Response To Human
Action Input: "your response to the human, summarizing what you did and what you learned"

Begin!
"""

def get_streaming_response(messages):
    completion = client.chat.completions.create(
        model="llama-3.2-11b-text-preview",
        messages=messages,
        temperature=0,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    full_response = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        full_response += content
        print(content, end="")
    print()  # New line after response
    return full_response

def stream_agent(prompt):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    def extract_action_and_input(text):
        action_pattern = r"Action: (.+?)\n"
        input_pattern = r"Action Input: \"(.+?)\""
        action = re.findall(action_pattern, text)
        action_input = re.findall(input_pattern, text)
        return action, action_input
    
    while True:
        response_text = get_streaming_response(messages)
        
        action, action_input = extract_action_and_input(response_text)
        
        if not action or not action_input:
            break
            
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Calculator":
            tool = calculator
        elif action[-1] == "Response To Human":
            break
            
        observation = tool(action_input[-1])
        print("Observation: ", observation)
        
        messages.extend([
            {"role": "assistant", "content": response_text},
            {"role": "user", "content": f"Observation: {observation}"},
        ])

# Example usage
if __name__ == "__main__":
    stream_agent("What was the high temperature in New York City yesterday?")

Thought: I need to find the high temperature in New York City yesterday.
Action: Search
Action Input: "high temperature in New York City yesterday"

Please wait for the human's observation.
Observation:  (Weather station: New York City - Central Park, USA). ... New York Temperature Yesterday. Maximum temperature yesterday: 68 °F (at 2:51 pm) ... Average temperature yesterday: 62 °F. High & Low Weather Summary for the Past Weeks Temperature Humidity Pressure; High: 84 °F (Sep 19, 2:51 pm) 94% (Sep 26, 2:51 pm) 30.19 "Hg (Sep 26, 2:51 pm) Low: New York, NY past weather data including previous temperature, barometric pressure, humidity, dew point, rain total, and wind conditions. ... High: 62.06ºf @12:51 AM Low: 60.98ºf @2:51 AM Approx. Precipitation / Rain Total: in. ... New York past weather yesterday along with historical conditions for the last 30 days, including previous high ... The National Weather Service Forecast Office in New York, NY provides official forecasts and warnings for

In [17]:
from groq import Groq
from duckduckgo_search import DDGS
from py_expression_eval import Parser
import re, time
from datetime import datetime

# Initialize Groq client
client = Groq()

# Initialize calculator
parser = Parser()

# DuckDuckGo search function
def search(search_term):
    search_result = ""
    with DDGS() as ddgs:
        results = ddgs.text(search_term, max_results=5)
        for result in results:
            search_result += result['body'] + " "
    return search_result

# Calculator function
def calculator(str):
    return parser.parse(str).evaluate({})

# DateTime function
def get_current_datetime(format_string=None):
    current = datetime.now()
    if format_string:
        try:
            return current.strftime(format_string)
        except ValueError:
            return str(current)
    return str(current)

# Get current date and time for system prompt
CURRENT_DATETIME = get_current_datetime()

# System prompt
SYSTEM_PROMPT = f"""
You are an AI assistant with access to current information. The current date and time is: {CURRENT_DATETIME}

You have access to the following tools:

1. Search: Useful for when you need to answer questions about current events. You should ask targeted questions.
2. Calculator: Useful for when you need to answer questions about math. Use python code, eg: 2 + 2
3. DateTime: Useful for when you need to know the current date or time in a specific format. You can provide a format string like "%Y-%m-%d" or "%H:%M:%S".
4. Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of two things:

Option 1: You use a tool to answer the question.
For this, you should use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator, DateTime]
Action Input: "the input to the action, to be sent to the tool"

After this, the human will respond with an observation, and you will continue.

Option 2: You respond to the human.
For this, you should use the following format:
Action: Response To Human
Action Input: "your response to the human, summarizing what you did and what you learned"

Remember: You already know the current date and time is {CURRENT_DATETIME}, but you can use the DateTime tool if you need it in a different format or need to verify the current time again.

Begin!
"""

def get_streaming_response(messages):
    completion = client.chat.completions.create(
        model="llama-3.2-11b-text-preview",
        messages=messages,
        temperature=0,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    full_response = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        full_response += content
        print(content, end="")
    print()  # New line after response
    return full_response

def stream_agent(prompt):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    def extract_action_and_input(text):
        action_pattern = r"Action: (.+?)\n"
        input_pattern = r"Action Input: \"(.+?)\""
        action = re.findall(action_pattern, text)
        action_input = re.findall(input_pattern, text)
        return action, action_input
    
    while True:
        response_text = get_streaming_response(messages)
        
        action, action_input = extract_action_and_input(response_text)
        
        if not action or not action_input:
            break
            
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Calculator":
            tool = calculator
        elif action[-1] == "DateTime":
            tool = get_current_datetime
        elif action[-1] == "Response To Human":
            break
            
        observation = tool(action_input[-1])
        print("Observation: ", observation)
        
        messages.extend([
            {"role": "assistant", "content": response_text},
            {"role": "user", "content": f"Observation: {observation}"},
        ])

# Example usage
if __name__ == "__main__":
    # test_queries = [
    #     "What day of the week will it be 3 days from now?",
    #     "How many hours until midnight?",
    #     "What's the current temperature in New York City?"
    # ]
    
    # for query in test_queries:
    #     print(f"\nQuery: {query}")
    #     stream_agent(query)
    #     time.sleep(2)  # Pause between queries
    stream_agent("""How many r in starberry?""")

Thought: I need to count the number of 'r's in the word 'starberry'
Action: Calculator
Action Input: 1 + 1 + 1 + 1 + 1 

However, I can do this more efficiently by using a single calculation with the correct number of 'r's. The word 'starberry' has 2 'r's. So the correct calculation is: 2 * 1 

Here is the corrected action:
Action: Calculator
Action Input: 2 * 1
